## 1. Notebook for predicting habit of lot of images
* loop through images - predict most likely habit + all habit possibilities
* save to csv

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import register_keras_serializable
import csv
from glob import glob
import os
import pandas as pd
import concurrent.futures

2024-10-31 12:38:26.568827: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
## where is the data
# Jaffeux data -> is the training data used for the model
Jaffeux_data = '/gws/nopw/j04/dcmex/users/ezriab/cp-of-Jaffeux-22/DS_training_data/' 
j_category = 'class_05_CG/' #class_01_Pri  class_02_Co  class_03_CC  class_04_ABC  class_05_CG  class_06_HC  class_07_Di  class_08_FA  class_09_WD
#dcmex_2ds = '/gws/nopw/j04/dcmex/users/ezriab/2dprocessed/flight_220730153000/'
dcmex_2ds = '/gws/nopw/j04/dcmex/users/ezriab/processed_images/2ds/ch_0/220730153000/'

file_path = dcmex_2ds # path to all files

# get all files in specif folder
file_list = glob(file_path+'*.png') # whole string of path + filenames in location
file_names = [os.path.basename(file_path) for file_path in file_list] # just file names


## where would you like predictions saved?
#save_loc = '/gws/nopw/j04/dcmex/users/ezriab/2dprocessed/flight_220730153000/'
save_loc = '/gws/nopw/j04/dcmex/users/ezriab/processed_images/2ds/ch_0/220730153000/'
save_name = 'habit_predictions.csv'

## quite important
categories = ['CA', 'Co',  'CC', 'CBC', 'CG', 'HPC', 'Dif', 'FA', 'WD']

In [3]:
## open model 
model_path = '/gws/nopw/j04/dcmex/users/ezriab/cp-of-Jaffeux-22/novmodel.h5py' # to h5 folder

# Define custom layer with serialization support
# (model would not open without this)
@register_keras_serializable()
class RandomFlip(tf.keras.layers.Layer):
    def __init__(self, mode='horizontal_and_vertical', seed=None, **kwargs):
        super(RandomFlip, self).__init__(**kwargs)
        self.mode = mode
        self.seed = seed

    def call(self, inputs, training=None):
        if training:
            if self.mode == 'horizontal_and_vertical':
                return tf.image.random_flip_left_right(tf.image.random_flip_up_down(inputs), seed=self.seed)
            elif self.mode == 'horizontal':
                return tf.image.random_flip_left_right(inputs, seed=self.seed)
            elif self.mode == 'vertical':
                return tf.image.random_flip_up_down(inputs, seed=self.seed)
        return inputs

    def get_config(self):
        config = super(RandomFlip, self).get_config()
        config.update({'mode': self.mode, 'seed': self.seed})
        return config

# Load the model with custom objects
custom_objects = {'RandomFlip': RandomFlip}
model = load_model(model_path, custom_objects=custom_objects)

## may also want to see summary of model -> check its doing correct things
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 200, 200, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 198, 198, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 99, 99, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 128)       0         
 g2D)                                                   

In [ ]:
# list to store results
## this was ran the first time 

for i in range(len(file_names)):
    
    img = image.load_img(file_path+file_names[i], target_size=(200, 200), color_mode='grayscale')
    #img = image.load_img(file_path+file_names[i])#, target_size=(200, 200), color_mode='grayscale')
    
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Predict using the loaded model
    predictions = model.predict(img_array)[0]
    # Find the index of the category with the highest probability
    predicted_index = np.argmax(predictions)
    
    # Get the corresponding category
    predicted_category = categories[predicted_index]

    # Prepare the data to be saved
    data = {'Name': file_names[i][:-4], 'Category': predicted_category}
    for i, category in enumerate(categories):
        data[category] = predictions[i]
    
    # Append the data to results list
    results.append(data)

df = pd.DataFrame(results, columns=columns_names)
# Save the DataFrame to a CSV file
df.to_csv(save_loc+save_name, index=False)
print('Finished')

### something going wrong

In [ ]:
# List to store results
## something is going wrong on new images? so have try + except + new function

# Define a function to load an image
def load_image_with_timeout(file_path, target_size=(200, 200), color_mode='grayscale', timeout=10):
    try:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(image.load_img, file_path, target_size=target_size, color_mode=color_mode)
            img = future.result(timeout=timeout)  # Set a timeout in seconds
        return img
    except concurrent.futures.TimeoutError:
        print(f"Skipping {file_path}: loading timed out.")
    except Exception as e:
        print(f"Skipping {file_path}: error - {e}")
    return None


results = []

for i in range(len(file_names)):
    try:
        # Load the image
        print(file_names[i])
        
        # Attempt to load the image with a timeout
        img = load_image_with_timeout(file_path + file_names[i])
        if img is None:
            continue  # Skip this image if it couldn't be loaded
            
        '''#img = image.load_img(file_path + file_names[i], target_size=(200, 200), color_mode='grayscale')'''
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Predict using the loaded model
        predictions = model.predict(img_array)[0]  # Set verbose=0 to reduce output
        predicted_index = np.argmax(predictions)
        predicted_category = categories[predicted_index]

        # Prepare the data to be saved
        data = {'Name': file_names[i][:-4], 'Category': predicted_category}
        for j, category in enumerate(categories):
            data[category] = predictions[j]

        # Append the data to results list
        results.append(data)

    except Exception as e:
        print(f"Error processing {file_names[i]}: {e}")

# Save the results to a DataFrame and CSV
df = pd.DataFrame(results, columns=columns_names)
df.to_csv(save_loc + save_name, index=False)
print('Finished')

1000081_1.png
1/1 [==============================] - 0s 298ms/step
1001384_1.png
1/1 [==============================] - 0s 123ms/step
1002210_1.png
1/1 [==============================] - 0s 63ms/step
1002353_1.png
1/1 [==============================] - 0s 56ms/step
1003804_1.png
1/1 [==============================] - 0s 143ms/step
1003990_2.png
1/1 [==============================] - 0s 55ms/step
1007977_1.png
1/1 [==============================] - 0s 103ms/step
1008727_3.png
1/1 [==============================] - 0s 143ms/step
1008795_1.png
1/1 [==============================] - 0s 125ms/step
1009039_1.png
1/1 [==============================] - 0s 122ms/step
1009089_1.png
1/1 [==============================] - 0s 53ms/step
1024549_1.png
1/1 [==============================] - 0s 131ms/step
1029769_1.png
1/1 [==============================] - 0s 50ms/step
1030597_1.png
1/1 [==============================] - 0s 133ms/step
1031613_1.png
1/1 [==============================] - 1s 527ms/step
